In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', 26)

from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()

import datetime as dt

In [2]:
train = pd.read_excel('Train/train_Data.xlsx')
test = pd.read_excel('test_Data.xlsx')
print(train.head())
print(test.head())

   ID Frequency InstlmentMode LoanStatus   PaymentMode  BranchID      Area  \
0   1   Monthly        Arrear     Closed         PDC_E         1       NaN   
1   2   Monthly       Advance     Closed           PDC       333    BHOPAL   
2   3   Quatrly        Arrear     Active  Direct Debit         1       NaN   
3   7   Monthly       Advance     Closed        Billed       125      GUNA   
4   8   Monthly        Arrear     Closed        Billed       152  BILASPUR   

   Tenure  AssetCost  AmountFinance  DisbursalAmount      EMI DisbursalDate  \
0      48     450000       275000.0         275000.0  24000.0    2012-02-10   
1      47     485000       350000.0         350000.0  10500.0    2012-03-31   
2      68     690000       519728.0         519728.0  38300.0    2017-06-17   
3      48     480000       400000.0         400000.0  11600.0    2013-11-29   
4      44     619265       440000.0         440000.0  15000.0    2011-12-08   

  MaturityDAte   AuthDate   AssetID  ManufacturerID  Sup

In [3]:
submission = pd.read_csv('sample_submission_ejm25Dc.csv')
submission.head()

ID  Top-up Month
0    4  36-48 Months
1    5  18-24 Months
2    6  30-36 Months
3   25  30-36 Months
4  119  30-36 Months

In [4]:
train_shape = train.shape
test_shape = test.shape

In [5]:
print("Shape of Train :", train.shape)
print("Shape of Test :", test.shape)
print("Shape of Submission :", submission.shape)

Shape of Train : (128655, 26)
Shape of Test : (14745, 25)
Shape of Submission : (14745, 2)


In [6]:
target = train[['Top-up Month']]

In [7]:
data = pd.concat((train, test)).reset_index(drop=True)
data.drop(['Top-up Month'], axis=1, inplace=True)
data.head()

C:\Users\anmol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


AGE  AmountFinance      Area  AssetCost   AssetID   AuthDate  BranchID  \
0  49.0       275000.0       NaN     450000   4022465 2012-02-10         1   
1  23.0       350000.0    BHOPAL     485000   4681175 2012-03-31       333   
2  39.0       519728.0       NaN     690000  25328146 2017-06-17         1   
3  24.0       400000.0      GUNA     480000  13021591 2013-11-29       125   
4  56.0       440000.0  BILASPUR     619265   3291320 2011-12-08       152   

          City  DisbursalAmount DisbursalDate      EMI Frequency  ID  \
0       RAISEN         275000.0    2012-02-10  24000.0   Monthly   1   
1       SEHORE         350000.0    2012-03-31  10500.0   Monthly   2   
2       BHOPAL         519728.0    2017-06-17  38300.0   Quatrly   3   
3  ASHOK NAGAR         400000.0    2013-11-29  11600.0   Monthly   7   
4     BILASPUR         440000.0    2011-12-08  15000.0   Monthly   8   

  InstlmentMode    LTV LoanStatus  ManufacturerID MaturityDAte  MonthlyIncome  \
0        Arrear  61.11     Closed          1568.0   2016-01-15       35833.33   
1       Advance  70.00     Closed          1062.0   2016-02-15         666.67   
2        Arrear  69.77     Active          1060.0   2023-02-10       45257.00   
3       Advance  80.92     Closed          1060.0   2017-11-10       20833.33   
4        Arrear  71.05     Closed          1046.0   2015-07-05       27313.67   

    PaymentMode SEX           State  SupplierID  Tenure   ZiPCODE  
0         PDC_E   M  MADHYA PRADESH       21946      48  464993.0  
1           PDC   M  MADHYA PRADESH       34802      47  466001.0  
2  Direct Debit   M  MADHYA PRADESH      127335      68  462030.0  
3        Billed   M  MADHYA PRADESH       25094      48  473335.0  
4        Billed   M     CHATTISGARH       21853      44  495442.0

In [8]:
data.shape

(143400, 25)

In [9]:
def details(df):
    a = pd.DataFrame()
    a['Count'] = df.count()
    a['Null values'] = df.isnull().sum()
    a['Data types'] = df.dtypes
    a['No. of Unique values'] = df.nunique()
    return(a)

In [10]:
details(data).T

AGE AmountFinance    Area AssetCost AssetID  \
Count                  143334        143400  131201    143400  143400   
Null values                66             0   12199         0       0   
Data types            float64       float64  object     int64   int64   
No. of Unique values       73         22183     103      8168  143400   

                            AuthDate BranchID    City DisbursalAmount  \
Count                         143400   143400  132144          143400   
Null values                        0        0   11256               0   
Data types            datetime64[ns]    int64  object         float64   
No. of Unique values            2713      214     514           21061   

                       DisbursalDate      EMI Frequency      ID InstlmentMode  \
Count                         143400   143400    143400  143400        143400   
Null values                        0        0         0       0             0   
Data types            datetime64[ns]  float64    object   int64        object   
No. of Unique values            2847    25912         4  143400             2   

                          LTV LoanStatus ManufacturerID    MaturityDAte  \
Count                  143400     143400         143398          143399   
Null values                 0          0              2               1   
Data types            float64     object        float64  datetime64[ns]   
No. of Unique values     8070          2             13             982   

                     MonthlyIncome PaymentMode     SEX   State SupplierID  \
Count                       143152      143400  143336  143400     143400   
Null values                    248           0      64       0          0   
Data types                 float64      object  object  object      int64   
No. of Unique values         12271          11       2      22       4971   

                      Tenure  ZiPCODE  
Count                 143400   143027  
Null values                0      373  
Data types             int64  float64  
No. of Unique values     142     9908

In [11]:
data.describe(include='all').T

count unique                  top    freq  \
AGE              143334    NaN                  NaN     NaN   
AmountFinance    143400    NaN                  NaN     NaN   
Area             131201    103              LUCKNOW    9547   
AssetCost        143400    NaN                  NaN     NaN   
AssetID          143400    NaN                  NaN     NaN   
AuthDate         143400   2713  2017-10-31 00:00:00    1438   
BranchID         143400    NaN                  NaN     NaN   
City             132144    514             NALGONDA    2483   
DisbursalAmount  143400    NaN                  NaN     NaN   
DisbursalDate    143400   2847  2017-10-31 00:00:00    1429   
EMI              143400    NaN                  NaN     NaN   
Frequency        143400      4          Half Yearly   87101   
ID               143400    NaN                  NaN     NaN   
InstlmentMode    143400      2               Arrear  136586   
LTV              143400    NaN                  NaN     NaN   
LoanStatus       143400      2               Closed  105723   
ManufacturerID   143398    NaN                  NaN     NaN   
MaturityDAte     143399    982  2018-06-10 00:00:00    2788   
MonthlyIncome    143152    NaN                  NaN     NaN   
PaymentMode      143400     11                  ECS   35143   
SEX              143336      2                    M  136273   
State            143400     22       MADHYA PRADESH   22495   
SupplierID       143400    NaN                  NaN     NaN   
Tenure           143400    NaN                  NaN     NaN   
ZiPCODE          143027    NaN                  NaN     NaN   

                               first                 last         mean  \
AGE                              NaN                  NaN      40.6205   
AmountFinance                    NaN                  NaN       351150   
Area                             NaN                  NaN          NaN   
AssetCost                        NaN                  NaN       557602   
AssetID                          NaN                  NaN  1.64289e+07   
AuthDate         2010-02-03 00:00:00  2019-11-23 00:00:00          NaN   
BranchID                         NaN                  NaN       169.69   
City                             NaN                  NaN          NaN   
DisbursalAmount                  NaN                  NaN       350779   
DisbursalDate    2010-02-01 00:00:00  2019-11-23 00:00:00          NaN   
EMI                              NaN                  NaN      55467.3   
Frequency                        NaN                  NaN          NaN   
ID                               NaN                  NaN      71700.5   
InstlmentMode                    NaN                  NaN          NaN   
LTV                              NaN                  NaN      59.3455   
LoanStatus                       NaN                  NaN          NaN   
ManufacturerID                   NaN                  NaN      1166.99   
MaturityDAte     2010-11-15 00:00:00  2059-08-10 00:00:00          NaN   
MonthlyIncome                    NaN                  NaN      49784.5   
PaymentMode                      NaN                  NaN          NaN   
SEX                              NaN                  NaN          NaN   
State                            NaN                  NaN          NaN   
SupplierID                       NaN                  NaN      56381.2   
Tenure                           NaN                  NaN      42.7199   
ZiPCODE                          NaN                  NaN       424879   

                        std     min          25%          50%          75%  \
AGE                 11.7022      18           31           40           49   
AmountFinance        105455   50000       300000       350000       416960   
Area                    NaN     NaN          NaN          NaN          NaN   
AssetCost            107049  200000       500000       555000       616000   
AssetID          9.3329e+06  422271  1.00197e+07  1.50741e+07  2

In [12]:
data['Frequency'].unique()

array(['Monthly', 'Quatrly', 'Half Yearly', 'BI-Monthly'], dtype=object)

In [13]:
data['Frequency'] = data['Frequency'].map({'Monthly':0, 'Quatrly':1, 'Half Yearly':2, 'BI-Monthly':3})

In [14]:
data['InstlmentMode'].unique()

array(['Arrear', 'Advance'], dtype=object)

In [15]:
data['InstlmentMode'] = data['InstlmentMode'].map({'Arrear':0, 'Advance':1})

In [16]:
data['LoanStatus'].unique()

array(['Closed', 'Active'], dtype=object)

In [17]:
data['LoanStatus'] = data['LoanStatus'].map({'Closed':0, 'Active':1})

In [18]:
data['PaymentMode'].unique()

array(['PDC_E', 'PDC', 'Direct Debit', 'Billed', 'ECS', 'Auto Debit',
       'SI Reject', 'ECS Reject', 'Cheque', 'PDC Reject', 'Escrow'],
      dtype=object)

In [19]:
data['PaymentMode'] = LE.fit_transform(data['PaymentMode'])

In [20]:
data['BranchID'].nunique()

214

In [21]:
data['BranchID'] = data['BranchID'].astype(str)

In [22]:
data['BranchID'] = LE.fit_transform(data['BranchID'])

In [23]:
data[data['MaturityDAte'].isnull()]

AGE  AmountFinance      Area  AssetCost  AssetID   AuthDate  BranchID  \
53392  43.0       400000.0  MANDSAUR     495000  9510810 2013-03-16        67   

           City  DisbursalAmount DisbursalDate  EMI  Frequency     ID  \
53392  MANDSAUR         400000.0    2013-03-15  0.0          0  57606   

       InstlmentMode    LTV  LoanStatus  ManufacturerID MaturityDAte  \
53392              0  80.81           0          1046.0          NaT   

       MonthlyIncome  PaymentMode SEX           State  SupplierID  Tenure  \
53392       41666.67            1   M  MADHYA PRADESH       55150      45   

        ZiPCODE  
53392  458880.0

In [24]:
data['MaturityDAte'].fillna(pd.to_datetime('2016-12-15'), inplace=True)

In [25]:
data['ManufacturerID'].nunique()

13

In [26]:
data[data['ManufacturerID'].isnull()]

AGE  AmountFinance     Area  AssetCost   AssetID   AuthDate  \
129088  32.0       385000.0     GUNA     520000  12486458 2013-09-06   
141339  58.0       480000.0  LUCKNOW     650000  14837714 2015-01-23   

        BranchID         City  DisbursalAmount DisbursalDate      EMI  \
129088         9  ASHOK NAGAR         385000.0    2013-09-06  67200.0   
141339        65       HARDOI         480000.0    2015-01-23  24400.0   

        Frequency      ID  InstlmentMode    LTV  LoanStatus  ManufacturerID  \
129088          2    4394              0  61.12           0             NaN   
141339          0  116966              0  70.09           1             NaN   

       MaturityDAte  MonthlyIncome  PaymentMode SEX           State  \
129088   2017-06-10       31800.00            7   M  MADHYA PRADESH   
141339   2017-01-10       16666.67            9   M   UTTAR PRADESH   

        SupplierID  Tenure   ZiPCODE  
129088       23156      48  473443.0  
141339       98657      24  241001.0

In [27]:
data['ManufacturerID'].fillna(0, inplace=True)

In [28]:
data['ManufacturerID'] = data['ManufacturerID'].astype(str)

In [29]:
data['ManufacturerID'] = LE.fit_transform(data['ManufacturerID'])

In [30]:
data['SupplierID'].nunique()

4971

In [31]:
data['SupplierID'] = data['SupplierID'].astype(str)

In [32]:
data['SupplierID'] = LE.fit_transform(data['SupplierID'])

In [33]:
data['SEX'].unique()

array(['M', nan, 'F'], dtype=object)

In [34]:
data['SEX'] = data['SEX'].fillna(data['SEX'].mode()[0])

In [35]:
data['SEX'].unique()

array(['M', 'F'], dtype=object)

In [36]:
data['SEX'] = data['SEX'].map({'M':0, 'F':1})

In [37]:
data['AGE']=data['AGE'].fillna(data['AGE'].median())

In [38]:
data['AGE'] = data['AGE'].astype(int)

In [39]:
data['MonthlyIncome']=data['MonthlyIncome'].fillna(0)

In [40]:
data['State'] = LE.fit_transform(data['State'])

In [41]:
data['ZiPCODE'].nunique()

9908

In [42]:
data[data['ZiPCODE'].isnull()]

AGE  AmountFinance                          Area  AssetCost   AssetID  \
246      33       300000.0                       BURDWAN     557976   1486208   
247      19       290000.0                       BURDWAN     491998   1501058   
248      41       250000.0                       BURDWAN     539092   1517344   
249      51       100000.0                       BURDWAN     522092   1561868   
256      28       300000.0                       BURDWAN     525442   1599023   
258      32       350000.0                       BURDWAN     621313   1615023   
259      24       200000.0                           NaN     506896   1620506   
645      43       200000.0                        KANPUR     349500   1623214   
646      24       200000.0                        KANPUR     343000   1680543   
740      28       200000.0                       LUCKNOW     475000   1686415   
852      27       350000.0            AHMEDABAD AMBAVADI     425000   1621152   
981      40       627000.0                    GANDHIDHAM     860000    841003   
1062     39       200000.0                           NaN     430000   1590402   
1139     45       800000.0                    VIJAYAWADA    1310560   1578511   
1192     22       325000.0  LUDHIANA FEROZ GANDHI MARKET     465000   1479975   
1328     38       100000.0                           NaN     265000   1591691   
1444     41       400000.0                        KARNAL     570000   1575483   
1445     26       350000.0                        KARNAL     545000   1667821   
1467     24       250000.0                    CHANDIGARH     400000    496923   
1511     31       280000.0                    CHANDIGARH     450000   1489971   
1512     25       280000.0                    CHANDIGARH     425000   1527830   
1513     53       350000.0                    CHANDIGARH     480000   1586208   
1514     30       300000.0                        KARNAL     435000   1588063   
1515     44       200000.0                    CHANDIGARH     330000   1604611   
1517     50       350000.0                    CHANDIGARH     460000   1605223   
1520     29       350000.0                        KARNAL     550000   1708663   
1674     29       190000.0                         SIRSA     285000   1627731   
2307     47       250000.0                           NaN     450000   1557881   
2313     25       185000.0                           NaN     460000   1584624   
2339     58       325000.0                           NaN     475000   1637201   
...     ...            ...                           ...        ...       ...   
60511    27       200000.0                      GULBARGA     598000   1603968   
60515    40       300000.0                      GULBARGA     638188   1684792   
61088    37       150000.0                   HOSHANGABAD     401000    462728   
61641    22       320000.0                   HOSHANGABAD     440000   1470230   
61644    38       200000.0                   HOSHANGABAD     597785   1506051   
61651    53       250000.0                   HOSHANGABAD     486000   1556821   
61653    34       200000.0                   HOSHANGABAD     415000   1573600   
61658    36       200000.0                   HOSHANGABAD     445000   1579105   
61664    31       325000.0                   HOSHANGABAD     545785   1591547   
61673    29       200000.0                   HOSHANGABAD     505000   1637467   
61701    23       200000.0                   HOSHANGABAD     605000   1769605   
62774    33       400000.0                     NIZAMABAD     623000   1638969   
65519    66       300000.0                           NaN     481000   1591498   
65520    28       302580.0                           NaN     498541   1604467   
65522    44       300000.0                           NaN     421000   1669828   
67797    41       269000.0                         PATNA     508185   1558036   
67798    38       250000.0                         PATNA     458525   1568451   
67799    28       250000.0             

In [43]:
data['ZiPCODE'] = data.groupby('BranchID')['ZiPCODE'].transform(lambda x: x.fillna(x.mode()[0]))

In [44]:
data['ZiPCODE'] = data['ZiPCODE'].astype(str)

In [45]:
data['ZiPCODE'] = LE.fit_transform(data['ZiPCODE'])

In [46]:
diff = pd.DataFrame()

In [47]:
diff['Amount'] = data['AmountFinance']-data['DisbursalAmount']
diff[diff['Amount'] != 0]

Amount
112      62125.49
478      15311.55
921      30994.73
1730     38097.38
1731     25453.29
1851     94518.22
1889     31170.90
2311     60910.67
2479     23589.63
2578     83289.83
2616     62173.33
2772     58292.58
2856    121933.83
2915     74970.70
2942    142537.62
3018     81798.17
3020    162186.72
3053     76624.67
3128    111185.93
3186     87187.00
3252    131578.00
3259    108352.00
3302     79668.65
3803     50270.00
3808     65879.17
3832     77586.61
3919     92290.00
3953     79086.00
3966     94086.00
4114     22387.49
...           ...
139000   11735.98
139001   11571.00
139006   19814.36
139011   24357.80
139044   22000.00
139211   14950.00
139262   15792.00
139516   29474.29
139574   12720.92
140515   34936.40
140561   54000.00
140623   70732.22
140978   24995.36
141284   26659.70
141307   39708.14
141702   30208.80
141707   13430.95
141897   24146.89
142155   29162.28
142248   24709.77
142287   15163.68
142367   21908.00
142553      39.00
142693   32183.14
142709   10100.00
142762   20846.90
143014   44131.27
143105    8925.00
143108   17659.00
143175   10600.00

[1552 rows x 1 columns]

In [48]:
diff['Date'] = data['DisbursalDate']-data['AuthDate']
diff[diff['Date'] != '0 days']

Amount    Date
33          0.0 -1 days
39          0.0 -3 days
42          0.0 -1 days
43          0.0 -1 days
47          0.0 -1 days
51          0.0 -1 days
52          0.0 -3 days
54          0.0 -2 days
55          0.0 -2 days
56          0.0 -1 days
57          0.0 -2 days
58          0.0 -1 days
59          0.0 -2 days
130         0.0 -1 days
135         0.0 -1 days
174         0.0  1 days
182         0.0  1 days
190         0.0  1 days
329         0.0 -2 days
357         0.0 -1 days
379         0.0 -1 days
380         0.0 -1 days
399         0.0 -1 days
401         0.0 -1 days
405         0.0 -1 days
406         0.0 -1 days
407         0.0 -1 days
409         0.0 -1 days
410         0.0 -1 days
432         0.0 -1 days
...         ...     ...
143165      0.0 -1 days
143167      0.0 -1 days
143175  10600.0 -1 days
143179      0.0 -3 days
143180      0.0 -1 days
143193      0.0 -1 days
143214      0.0 -1 days
143215      0.0 -1 days
143216      0.0 -1 days
143230      0.0 -1 days
143243      0.0 -1 days
143244      0.0 -1 days
143248      0.0 -1 days
143265      0.0 -1 days
143288      0.0 -1 days
143292      0.0 -1 days
143297      0.0 -1 days
143300      0.0 -2 days
143312      0.0 -2 days
143327      0.0 -1 days
143328      0.0 -1 days
143329      0.0 -1 days
143353      0.0 -2 days
143359      0.0 -1 days
143362      0.0 -1 days
143366      0.0 -1 days
143368      0.0 -2 days
143388      0.0 -1 days
143398      0.0 -1 days
143399      0.0 -1 days

[18207 rows x 2 columns]

In [49]:
((data.isnull().sum())/(len(data)))*100

AGE                0.000000
AmountFinance      0.000000
Area               8.506974
AssetCost          0.000000
AssetID            0.000000
AuthDate           0.000000
BranchID           0.000000
City               7.849372
DisbursalAmount    0.000000
DisbursalDate      0.000000
EMI                0.000000
Frequency          0.000000
ID                 0.000000
InstlmentMode      0.000000
LTV                0.000000
LoanStatus         0.000000
ManufacturerID     0.000000
MaturityDAte       0.000000
MonthlyIncome      0.000000
PaymentMode        0.000000
SEX                0.000000
State              0.000000
SupplierID         0.000000
Tenure             0.000000
ZiPCODE            0.000000
dtype: float64

In [50]:
data.drop(['Area','City','AssetID'],axis=1,inplace=True)
data.head()

AGE  AmountFinance  AssetCost   AuthDate  BranchID  DisbursalAmount  \
0   49       275000.0     450000 2012-02-10         0         275000.0   
1   23       350000.0     485000 2012-03-31       128         350000.0   
2   39       519728.0     690000 2017-06-17         0         519728.0   
3   24       400000.0     480000 2013-11-29         9         400000.0   
4   56       440000.0     619265 2011-12-08        26         440000.0   

  DisbursalDate      EMI  Frequency  ID  InstlmentMode    LTV  LoanStatus  \
0    2012-02-10  24000.0          0   1              0  61.11           0   
1    2012-03-31  10500.0          0   2              1  70.00           0   
2    2017-06-17  38300.0          1   3              0  69.77           1   
3    2013-11-29  11600.0          0   7              1  80.92           0   
4    2011-12-08  15000.0          0   8              0  71.05           0   

   ManufacturerID MaturityDAte  MonthlyIncome  PaymentMode  SEX  State  \
0               8   2016-01-15       35833.33            9    0     12   
1               5   2016-02-15         666.67            7    0     12   
2               4   2023-02-10       45257.00            3    0     12   
3               4   2017-11-10       20833.33            1    0     12   
4               2   2015-07-05       27313.67            1    0      4   

   SupplierID  Tenure  ZiPCODE  
0        1991      48     4976  
1        2861      47     5017  
2         828      68     4916  
3        2492      48     5121  
4        1985      44     5475

In [51]:
data.dtypes

AGE                         int32
AmountFinance             float64
AssetCost                   int64
AuthDate           datetime64[ns]
BranchID                    int32
DisbursalAmount           float64
DisbursalDate      datetime64[ns]
EMI                       float64
Frequency                   int64
ID                          int64
InstlmentMode               int64
LTV                       float64
LoanStatus                  int64
ManufacturerID              int32
MaturityDAte       datetime64[ns]
MonthlyIncome             float64
PaymentMode                 int32
SEX                         int64
State                       int32
SupplierID                  int32
Tenure                      int64
ZiPCODE                     int32
dtype: object

In [52]:
data['MaturityDAte_day'] = data['MaturityDAte'].dt.day
data['MaturityDAte_month'] = data['MaturityDAte'].dt.month
data['MaturityDAte_year'] = data['MaturityDAte'].dt.year
data['AuthDate_day'] = data['AuthDate'].dt.day
data['AuthDate_month'] = data['AuthDate'].dt.month
data['AuthDate_year'] = data['AuthDate'].dt.year
data['DisbursalDate_day'] = data['DisbursalDate'].dt.day
data['DisbursalDate_month'] = data['DisbursalDate'].dt.month
data['DisbursalDate_year'] = data['DisbursalDate'].dt.year
data.drop(['MaturityDAte','AuthDate','DisbursalDate'], axis=1, inplace=True)

In [53]:
data.drop(['MaturityDAte','AuthDate','DisbursalDate'], axis=1, inplace=True)

In [54]:
data.head()

AGE  AmountFinance  AssetCost  BranchID  DisbursalAmount      EMI  \
0   49       275000.0     450000         0         275000.0  24000.0   
1   23       350000.0     485000       128         350000.0  10500.0   
2   39       519728.0     690000         0         519728.0  38300.0   
3   24       400000.0     480000         9         400000.0  11600.0   
4   56       440000.0     619265        26         440000.0  15000.0   

   Frequency  ID  InstlmentMode    LTV  LoanStatus  ManufacturerID  \
0          0   1              0  61.11           0               8   
1          0   2              1  70.00           0               5   
2          1   3              0  69.77           1               4   
3          0   7              1  80.92           0               4   
4          0   8              0  71.05           0               2   

   MonthlyIncome  ...  State  SupplierID  Tenure  ZiPCODE  MaturityDAte_day  \
0       35833.33  ...     12        1991      48     4976                15   
1         666.67  ...     12        2861      47     5017                15   
2       45257.00  ...     12         828      68     4916                10   
3       20833.33  ...     12        2492      48     5121                10   
4       27313.67  ...      4        1985      44     5475                 5   

   MaturityDAte_month  MaturityDAte_year  AuthDate_day  AuthDate_month  \
0                   1               2016            10               2   
1                   2               2016            31               3   
2                   2               2023            17               6   
3                  11               2017            29              11   
4                   7               2015             8              12   

   AuthDate_year  DisbursalDate_day  DisbursalDate_month  DisbursalDate_year  
0           2012                 10                    2                2012  
1           2012                 31                    3                2012  
2           2017                 17                    6                2017  
3           2013                 29                   11                2013  
4           2011                  8                   12                2011  

[5 rows x 28 columns]

In [55]:
train = data.iloc[:train_shape[0]]
test = data.iloc[train_shape[0]:]

In [56]:
print("Shape of Train :", train.shape)
print("Shape of Test :", test.shape)
print("Shape of Submission :", submission.shape)

Shape of Train : (128655, 28)
Shape of Test : (14745, 28)
Shape of Submission : (14745, 2)


In [57]:
train.dtypes

AGE                      int32
AmountFinance          float64
AssetCost                int64
BranchID                 int32
DisbursalAmount        float64
EMI                    float64
Frequency                int64
ID                       int64
InstlmentMode            int64
LTV                    float64
LoanStatus               int64
ManufacturerID           int32
MonthlyIncome          float64
PaymentMode              int32
SEX                      int64
State                    int32
SupplierID               int32
Tenure                   int64
ZiPCODE                  int32
MaturityDAte_day         int64
MaturityDAte_month       int64
MaturityDAte_year        int64
AuthDate_day             int64
AuthDate_month           int64
AuthDate_year            int64
DisbursalDate_day        int64
DisbursalDate_month      int64
DisbursalDate_year       int64
dtype: object